In [1]:
library(oce)  # 加载这个包用来计算太阳仰角以及离太阳的距离
library(plyr)

Loading required package: gsw

Loading required package: testthat



In [2]:
add_solar = function(data, longitude, latitude)
{
  cal_4_solar = function(time_str)
  {
    # 计算出太阳仰角， 离太阳的距离, 并根据大气折射进行校正
    result = sunAngle(as.POSIXct(time_str, tz="Asia/Taipei"), 
                      longitude = longitude, latitude = latitude, 
                      useRefraction = TRUE)
    
    # 计算出年份，月份，天数等等，作为特征
    time_list = as.POSIXlt(time_str, tz="Asia/Taipei")
    
    year = time_list$year
    month = time_list$mon + 1  # 月份居然从0开始
    yday = time_list$yday
    mday = time_list$mday
    hour = time_list$hour
    minute = time_list$min
    second = time_list$sec
    hours_float = hour + minute/60 + second/3600
    
    # 把计算出的结果统一存到result这个list里，然后返回
    result$time = NULL  # 这个浮点日期没啥意义，删掉
    result$hours_float = hours_float
    result$mday = mday
    result$yday = yday
    result$month = month
    result$year = year
    
    return(result)
  }
  
  # 把计算出的结果存一个dataframe
  pre_df = t(sapply(data$时间, cal_4_solar))
  
  solar_df = data.frame(matrix(unlist(pre_df), nrow=dim(pre_df)[1], ncol=dim(pre_df)[2], byrow=FALSE))
  colnames(solar_df) = colnames(pre_df)
  # 把计算出的特征dataframe和原始的dataframe合并起来, 然后返回
  cbind_df = cbind(data, solar_df)
  
  return(cbind_df)
}

In [3]:
latitude_list=list(36.3329,43.0287,36.3329,36.3329,43.0287,36.3329,43.0287,36.3329,36.6207,43.8016)
longtude_list=list(97.1849,92.6784,97.1849,97.1849,92.6784,97.1849,92.6784,97.1849,101.825,87.5967)

In [5]:
for(i in 1:10){
    setwd('J:/人工智能学习/数据集/【更新】国能日新竞赛数据/国能日新竞赛数据/test')
    sprintf('正在处理文件 %d',i) 
    file_name=paste('test_',i,'.csv',sep = "")
    train = read.csv(file_name, fileEncoding = 'UTF-8')
    longitude=longtude_list[[i]]
    latitude=latitude_list[[i]]
    train_new=add_solar(train,longitude,latitude)
    setwd('J:/人工智能学习/数据集/【更新】国能日新竞赛数据/国能日新竞赛数据/data_new')
    new_file=paste('test_new_',i,'.csv',sep="")
    write.csv(train_new, new_file, row.names = FALSE, fileEncoding = 'GBK')}